## Fetch post to delete

In [1]:
from atproto import Client
import re

def convert_web_url_to_at_uri(web_url):
    # Extract components from web URL
    pattern = r"https://bsky\.app/profile/([^/]+)/post/([^/]+)"
    match = re.match(pattern, web_url)
    if match:
        handle, post_id = match.groups()
        return f"at://{handle}/app.bsky.feed.post/{post_id}"
    return None

# Initialize client and login
client = Client()
client.login('ramith.fyi', '3mpu-vaed-7uvs-c4so')

# Convert web URL to AT URI
web_url = "https://bsky.app/profile/chadbrambley.bsky.social/post/3lc2muiuepk2v"
at_uri = convert_web_url_to_at_uri(web_url)

# Make the request with the AT URI
response = client.get_post_thread(uri=at_uri)
post_cid = response.thread.post.cid
post_text = response.thread.post.record.text

print(post_cid)
print(post_text)

bafyreidhfvbr53ipidqduwm4vdlpzaig5k5sdp4jao45xfq4v2o3swbqmu
Protein LMs, genomic LMs, and chemical LMs, to name only a few. Incredible potential to transform medicine (already are), and they're based on the same tech as any language model. And all are technically subsets of machine learning. So, not completely different, to be fair.


### Make a connection to the db and locate the timestamp

In [2]:
from datetime import datetime
import peewee

db = peewee.SqliteDatabase('feed_database2.db', pragmas={'journal_mode': 'wal'})

class BaseModel(peewee.Model):
    class Meta:
        database = db

class Post(BaseModel):
    uri = peewee.CharField(index=True)
    cid = peewee.CharField()
    reply_parent = peewee.CharField(null=True, default=None)
    reply_root = peewee.CharField(null=True, default=None)
    indexed_at = peewee.DateTimeField(default=datetime.utcnow)

# Use a connection context for safe reading
with db.connection_context():
    posts = Post.select()

    print(f"There are {len(posts)} posts")
    for post in posts:
        # print(f"URI: {post.uri}")

        if(post_cid == post.cid):
            print(f"Found CID: {post.cid} !!")
            print(f"Indexed At: {post.indexed_at}")

db.close()

There are 103 posts
Found CID: bafyreidhfvbr53ipidqduwm4vdlpzaig5k5sdp4jao45xfq4v2o3swbqmu !!
Indexed At: 2024-11-29 03:22:20.706371


False

In [3]:
post_cid

'bafyreidhfvbr53ipidqduwm4vdlpzaig5k5sdp4jao45xfq4v2o3swbqmu'

### Now delete that post

In [4]:
from datetime import datetime

# Reconfigure database with correct parameters
db = peewee.SqliteDatabase('feed_database2.db', pragmas={
    'journal_mode': 'wal',
    'cache_size': -1024 * 64
})

# Delete specific post by cid
target_cid = post_cid

with db.connection_context():
    deleted_count = Post.delete().where(Post.cid == target_cid).execute()
    print(f"Deleted {deleted_count} posts")

# If you want to be extra careful, you can check if it's closed
if not db.is_closed():
    print("closed db")
    db.close()

Deleted 1 posts
